# Energy models

This module is concerned with providing classes for computation of energy sources; these are mostly related to estimation of fuel consumption or integration to another simulation modules. Hereafter we illustrate the in cascading complexity the available energy sources. Some of these are simple wrapper around features provided by the combustion utilities.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from majordome import (
    StateType,
    NormalFlowRate,
    CombustionPowerOp,
    CombustionFlowOp,
    CombustionAtmosphereCHON,
    CombustionPowerSupply,
    HeatedGasEnergySource,
    CombustionEnergySource,
)

In [3]:
# import majordome.walang
# ct = __builtins__.cantera

## Combustion setup utilities

A common need in industry is to evaluate the real heating value from a gas composition reported by the supplier. This is the main goal of `CombustionAtmosphereCHON`, which has a very simple API provided below:

In [4]:
gas = CombustionAtmosphereCHON("gri30.yaml")
lhv = gas.solution_heating_value("CH4: 1", "O2: 1")
print(f"Solution heating value: {lhv:.2f} MJ/kg")

Solution heating value: 50.03 MJ/kg


Built upon it, `CombustionPowerSupply` performs the basic calculations for retrieving required flow rates from a given power specification; this proves very handy for the CFD engineer.

In [5]:
supply = CombustionPowerSupply(500.0, 1.0, "CH4: 1", "O2: 1", "gri30.yaml")
print(supply.report())

| Property                  | Unit   |    Value |
|---------------------------|--------|----------|
| Required power            | kW     | 500      |
| Lower heating value       | MJ/kg  |  50.0254 |
| Fuel mass flow rate       | kg/h   |  35.9817 |
| Oxidizer mass flow rate   | kg/h   | 143.532  |
| Total mass flow rate      | kg/h   | 179.514  |
| Fuel volume flow rate     | Nm³/h  |  50.2707 |
| Oxidizer volume flow rate | Nm³/h  | 100.541  |
| Total volume flow rate    | Nm³/h  | 150.812  |
| Water production          | kg/h   |  80.8092 |
| Carbon dioxide production | kg/h   |  98.7047 |
| Total emissions           | kg/h   | 179.514  |


**TODO:** missing a sample with `CombustionAtmosphereMixer`!

## Energy sources

In [6]:
source = HeatedGasEnergySource("airish.yaml", 500.0, mass_flow_rate=1.0)
print(source.report())

| Property               | Unit     | Value                 |
|------------------------|----------|-----------------------|
| Source kind            |          | HeatedGasEnergySource |
| Provided power         | kW       | 500.0                 |
| Source                 |          | airish.yaml           |
| Phase                  |          | air                   |
| Reference area         | m²       | 1.0                   |
| Mass flow rate         | kg/s     | 1.0                   |
| Volume flow rate       | m³/s     | 2.2051590808893957    |
| Momentum flux          | kg.m/s²  | 2.2051590808893957    |
| Reference temperature  | K        | 298.15                |
| Reference pressure     | Pa       | 101325.0              |
| Temperature            | K        | 778.5219246537463     |
| Pressure               | Pa       | 101324.99999999999    |
| Density                | kg/m³    | 0.453482022529039     |
| Specific enthalpy      | J/(kg.K) | 500038.49937969144    |
| Specif

In [7]:
source = HeatedGasEnergySource("airish.yaml", 500.0, mass_flow_rate=1.0,
                               cross_area=0.1, Y="N2: 0.79, O2: 0.21")
print(source.report())

| Property               | Unit     | Value                 |
|------------------------|----------|-----------------------|
| Source kind            |          | HeatedGasEnergySource |
| Provided power         | kW       | 500.0                 |
| Source                 |          | airish.yaml           |
| Phase                  |          | air                   |
| Reference area         | m²       | 0.1                   |
| Mass flow rate         | kg/s     | 1.0                   |
| Volume flow rate       | m³/s     | 2.2090693490835482    |
| Momentum flux          | kg.m/s²  | 22.090693490835484    |
| Reference temperature  | K        | 298.15                |
| Reference pressure     | Pa       | 101325.0              |
| Temperature            | K        | 774.4142430033143     |
| Pressure               | Pa       | 101324.99999999999    |
| Density                | kg/m³    | 0.45267931512193527   |
| Specific enthalpy      | J/(kg.K) | 500040.3236104384     |
| Specif

In [8]:
fuel_state = StateType("CH4: 1",             300, 101325)
oxid_state = StateType("N2: 0.79, O2: 0.21", 300, 101325)

In [9]:
ops = CombustionPowerOp(500.0, 1.0, fuel_state, oxid_state)
source = CombustionEnergySource("ch4/bfer.yaml", operation=ops, cross_area=0.1)

# Recover values for other tests that follow
# TODO wrap these in properties for clean API:
mdot_fuel = source._qty_fuel.mass
mdot_oxid = source._qty_oxid.mass

nfr_fuel = NormalFlowRate.new_from_solution(source._qty_fuel)
nfr_oxid = NormalFlowRate.new_from_solution(source._qty_oxid)

qdot_fuel = 3600 * mdot_fuel / nfr_fuel.density
qdot_oxid = 3600 * mdot_oxid / nfr_oxid.density

print(source.report())

| Property               | Unit     | Value                  |
|------------------------|----------|------------------------|
| Source kind            |          | CombustionEnergySource |
| Provided power         | kW       | 500.0                  |
| Source                 |          | ch4/bfer.yaml          |
| Phase                  |          | CH4_BFER               |
| Reference area         | m²       | 0.1                    |
| Mass flow rate         | kg/s     | 0.18117768922524702    |
| Volume flow rate       | m³/s     | 1.221027872548669      |
| Momentum flux          | kg.m/s²  | 2.2122300842798728     |
| ******* FLUE:          |          |                        |
| Temperature            | K        | 2257.7969133307774     |
| Pressure               | Pa       | 101325.00000819034     |
| Density                | kg/m³    | 0.14838128866548494    |
| Specific enthalpy      | J/(kg.K) | -254492.89776981465    |
| Specific heat capacity | J/(kg.K) | 1516.597922218732

In [10]:
ops = CombustionFlowOp("mass", mdot_fuel, mdot_oxid, fuel_state, oxid_state)
source = CombustionEnergySource("ch4/bfer.yaml", operation=ops, cross_area=0.1)
print(source.report())

| Property               | Unit     | Value                  |
|------------------------|----------|------------------------|
| Source kind            |          | CombustionEnergySource |
| Provided power         | kW       | 500.0                  |
| Source                 |          | ch4/bfer.yaml          |
| Phase                  |          | CH4_BFER               |
| Reference area         | m²       | 0.1                    |
| Mass flow rate         | kg/s     | 0.18117768922524702    |
| Volume flow rate       | m³/s     | 1.221027872548669      |
| Momentum flux          | kg.m/s²  | 2.2122300842798728     |
| ******* FLUE:          |          |                        |
| Temperature            | K        | 2257.7969133307774     |
| Pressure               | Pa       | 101325.00000819034     |
| Density                | kg/m³    | 0.14838128866548494    |
| Specific enthalpy      | J/(kg.K) | -254492.89776981465    |
| Specific heat capacity | J/(kg.K) | 1516.597922218732

In [11]:
ops = CombustionFlowOp("volume", qdot_fuel, qdot_oxid, fuel_state, oxid_state)
source = CombustionEnergySource("ch4/bfer.yaml", operation=ops, cross_area=0.1)
print(source.report())

| Property               | Unit     | Value                  |
|------------------------|----------|------------------------|
| Source kind            |          | CombustionEnergySource |
| Provided power         | kW       | 500.0                  |
| Source                 |          | ch4/bfer.yaml          |
| Phase                  |          | CH4_BFER               |
| Reference area         | m²       | 0.1                    |
| Mass flow rate         | kg/s     | 0.18117768922524705    |
| Volume flow rate       | m³/s     | 1.2210278725486692     |
| Momentum flux          | kg.m/s²  | 2.212230084279873      |
| ******* FLUE:          |          |                        |
| Temperature            | K        | 2257.7969133307774     |
| Pressure               | Pa       | 101325.00000819034     |
| Density                | kg/m³    | 0.14838128866548494    |
| Specific enthalpy      | J/(kg.K) | -254492.89776981465    |
| Specific heat capacity | J/(kg.K) | 1516.597922218732